In [1]:
import requests
import pandas as pd
from datetime import datetime
import os
import datetime
import time
import random

In [5]:
user = os.environ.get('reddit_username')
password = os.environ.get('reddit_password')
client_id = os.environ.get('reddit_clientid')
secret = os.environ.get('reddit_secret')
auth = requests.auth.HTTPBasicAuth(client_id, secret)

# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': user,
        'password': password}

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'meirlpostbot'}

# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# while the token is valid (~2 hours) we just add headers=headers to our requests
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

<Response [200]>

In [6]:
repeat = 10
subreddit = 'meirl'

posts = []
params = {'limit': 100,
          't': 'all'}
for i in range(repeat):
    res = requests.get('https://oauth.reddit.com/r/' + subreddit + '/top', headers=headers, params=params)
    for p in res.json()['data']['children']:
        posts.append(p['data'])
    params['after'] = 't3_' + posts[-1]['id']

In [7]:
posts = pd.DataFrame(posts)
postsDF = posts[['id', 'title', 'url', 'created_utc', 'is_video', 'ups']]
postsDF['months_old'] = (datetime.datetime.now().timestamp() - postsDF['created_utc'])/60/60/24/30

<ipython-input-7-ec0c912cf538>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  postsDF['months_old'] = (datetime.datetime.now().timestamp() - postsDF['created_utc'])/60/60/24/30


In [8]:
postsDF = postsDF.loc[postsDF['months_old'] > 6].tail(20)
postsDF = postsDF.loc[postsDF['is_video'] == False]
postsDF[['title', 'url']]

title                                  url
977  me_irl  https://i.redd.it/v8ol8ocz8bz11.jpg
978  Me irl      https://i.imgur.com/CR2ZLAO.jpg
979  me_irl  https://i.redd.it/vu723gg03c341.png
980   Meirl  https://i.redd.it/px6avcml60g21.jpg
981  Me Irl  https://i.redd.it/by8uxbzwws521.png
982   Meirl  https://i.redd.it/9myefxe9wtg11.jpg
983   meirl  https://i.redd.it/1xldzkapifq01.jpg
985   Meirl  https://i.redd.it/ehprogxde6h11.jpg
986  me irl       http://i.imgur.com/6EklLye.jpg
987   Meirl  https://i.redd.it/vq64tpwezgs01.jpg
988   meirl  https://i.redd.it/6vxvlu2h3wy11.jpg
989   meirl   https://i.redd.it/g1bu1kedksnz.jpg
990  me irl      https://i.imgur.com/nHwPhJJ.jpg
991   meirl  https://i.redd.it/yswprk536ec51.jpg
992  me irl  https://i.redd.it/uhp7vuy1mbm11.jpg
993   Meirl  https://i.redd.it/z0kr27mgtoa01.jpg
994  me irl  https://i.redd.it/dlcodononel11.jpg
995   meirl   https://i.redd.it/bvx4ifhugl5z.jpg
996  me📱irl   https://i.redd.it/j0j2io06az4y.jpg
997   meirl      https://i.imgur.com/2xoWedG.jpg

In [9]:
##imgur rehost
# imageUrl = postsDF['url'].iloc[0]
# imageTitle = postsDF['title'].iloc[0]
# #imgur_client_id = os.environ.get('imgur_clientid')
# #imgur_secret = os.environ.get('imgur_secret')
# #auth = requests.auth.HTTPBasicAuth(imgur_client_id, imgur_secret)
# data = {'type': 'url',
#         'image': imageUrl
#        }
# imageUrl = requests.post('https://api.imgur.com/3/upload', data = data).json()['data']['link']
# imageUrl

In [10]:
##reddit upload
#params = {
#     'sr': subreddit,
#     'kind': 'link',
#     'title': imageTitle,
#     'url': imageUrl
# }
# post = requests.post("https://oauth.reddit.com/api/submit",
#                        headers=headers,
#                        params=params)


In [11]:
lowMinute = 60
highMinute = 120

log = []
for i in postsDF.index:
    imageUrl = postsDF.loc[i]['url']
    imageTitle = postsDF.loc[i]['title']
    data = {'type': 'url',
            'image': imageUrl
           }
    imageUrl = requests.post('https://api.imgur.com/3/upload', data = data).json()['data']['link']
    params = {'sr': subreddit,
              'kind': 'link',
              'title': imageTitle,
              'url': imageUrl
             }
    post = requests.post("https://oauth.reddit.com/api/submit",
                       headers=headers,
                       params=params)
    log.append([i, imageUrl, imageTitle])
    print(i, imageUrl, imageTitle)
    sleeptime = random.randint(lowMinute * 60, highMinute * 60)
    print('sleep for', sleeptime/60, 'minutes')
    for i in range(sleeptime):
        time.sleep(1)

977 https://i.imgur.com/qsMaUYb.jpg me_irl
sleep for 241500 minute


KeyboardInterrupt: 